In [1]:
import glob, os, sys, toml, pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from multiprocessing import Pool, cpu_count
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor

from MOASMO_parameters import *
from MOASMO_parameter_allbasin_emulator import *

path_MOASMO = '/glade/u/home/guoqiang/CTSM_repos/ctsm_optz/MO-ASMO/src/'
sys.path.append(path_MOASMO)
import gp
import NSGA2


import os, sys, subprocess, time, toml
import pandas as pd
import numpy as np
from MOASMO_parameter_allbasin_emulator import allbasin_emulator_train_and_optimize, allbasin_emulator_CV_traintest_and_optimize
import run_multiple_paramsets_Derecho
from multiprocessing import Pool



In [2]:
    infile_basin_info = f"/glade/work/guoqiang/CTSM_CAMELS/data_mesh_surf/HillslopeHydrology/CAMELS_level1_basin_info.csv"
    infile_param_info = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/src/parameter/CTSM_CAMELS_SA_param_240202.csv'
    infile_attr_foruse = '/glade/u/home/guoqiang/CTSM_repos/CTSM_calibration/data/camels_attributes_table_TrainModel.csv'
    inpath_moasmo = "/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange"
    path_CTSM_case = f'/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_MOASMO_bigrange'
    ncpus = 36
    iterend = 1
    outpathname = 'LSE_spaceCV_PredictParam'
    suffix = 'LSEspaceCV'
    objfunc = 'normKGE'
    numruns = 10
    target_index = np.arange(627)

In [4]:
    cv_num = 5
    
    outpath = f"{inpath_moasmo}/{outpathname}"
    # os.makedirs(outpath, exist_ok=True)
    
    # Load data: same for all iterations
    df_basin_info = pd.read_csv(infile_basin_info)
    df_basin_info.index = np.arange(len(df_basin_info))
    all_index = np.arange(len(df_basin_info))

    # # divide into train/test index
    # outfile = f'{outpath}/train_test_CV_indices.npz'
    # if os.path.isfile(outfile):
    #     dtmp = np.load(outfile, allow_pickle=True)
    #     train_indices, test_indices = dtmp['train_indices'], dtmp['test_indices']
    # else:
    #     train_indices = []
    #     test_indices = []
    #     for i in range(cv_num):
    #         ind1 = all_index[i::cv_num]
    #         ind2 = np.setdiff1d(all_index, ind1)
    #         test_indices.append(ind1)
    #         train_indices.append(ind2)
    #         print('test index', ind1)
    #         print('train index', ind2)
    #     # np.savez_compressed(outfile, test_indices=np.array(test_indices, dtype=object), train_indices=np.array(train_indices, dtype=object))
    
    # # information for all basins
    # df_param_info = pd.read_csv(infile_param_info)

    # file_defa_param = f'{outpath}/camels_{len(all_index)}basin_ctsm_defa_param.csv'
    # df_param_defa = read_allbasin_defa_params(path_CTSM_case, infile_param_info, file_defa_param, all_index)

    # file_param_lb = f'{outpath}/camels_{len(all_index)}basin_ctsm_all_param_lb.gz'
    # file_param_ub = f'{outpath}/camels_{len(all_index)}basin_ctsm_all_param_ub.gz'
    # df_param_lb, df_param_ub = load_basin_param_bounds(inpath_moasmo, df_param_defa, file_param_lb, file_param_ub, all_index, suffix)

    file_camels_attribute = f'{outpath}/camels_{len(all_index)}basin_attribute.pkl'
    df_att = read_camels_attributes(infile_basin_info, file_camels_attribute, all_index)
    
    df_att_foruse = pd.read_csv(infile_attr_foruse)
    useattrs = list(df_att_foruse[df_att_foruse['att_Xie2021'].values]['Attribute_text'].values)
    print("The number of attributes used:", len(useattrs))
    print(useattrs)

    # # Load data: outputs from each iteration
    # for iter in range(0, iterend):
    #     file_all_param = f'{outpath}/camels_{len(all_index)}basin_ctsm_all_param_iter{iter}.gz'
    #     file_all_metric = f'{outpath}/camels_{len(all_index)}basin_ctsm_all_metric_iter{iter}.gz'
    #     file_all_basinid = f'{outpath}/camels_{len(all_index)}basin_ctsm_all_basinid_iter{iter}.gz'
        
    #     df_param_i, df_metric_i, df_basinid_i = load_all_basin_params_metrics(inpath_moasmo, df_param_defa, df_basin_info, iter, file_all_param, file_all_metric, file_all_basinid, all_index, suffix)
        
    #     df_basinid_i['iter'] = iter
        
    #     if iter == 0:
    #         df_param = df_param_i
    #         df_metric = df_metric_i
    #         df_basinid = df_basinid_i
    #     else:
    #         df_param = pd.concat([df_param, df_param_i])
    #         df_metric = pd.concat([df_metric, df_metric_i])
    #         df_basinid = pd.concat([df_basinid, df_basinid_i])
    
    # df_param.index = np.arange(len(df_param))
    # df_metric.index = np.arange(len(df_metric))
    # df_basinid.index = np.arange(len(df_basinid))

    # index = np.isnan(np.sum(df_metric.values, axis=1) + np.sum(df_param.values, axis=1))
    # df_param = df_param[~index]
    # df_metric = df_metric[~index]
    # df_basinid = df_basinid[~index]
    
    # df_param.index = np.arange(len(df_param))
    # df_metric.index = np.arange(len(df_metric))
    # df_basinid.index = np.arange(len(df_basinid))
    
    # print('Number of nan samples:', np.sum(index))
    # print("Number of original parameter sets:", len(index))
    # print("Number of final parameter sets:", len(df_param))

    # # Prepare model input and output
    # df_input = df_param.copy()
    # df_input["hru_id"] = df_basinid["hru_id"]
    # df_input = df_input.merge(df_att[useattrs + ["hru_id"]], on="hru_id", how="left")
    # df_input = df_input.drop(["hru_id"], axis=1)
    
    # inputnames = list(df_param.columns) + useattrs

    # # One-hot encoding for categorical attributes
    # for att in useattrs:
    #     if df_input[att].dtype == "object":
    #         print('Convert', att, 'to one-hot encoding')
    #         enc = OneHotEncoder(sparse=False)
    #         enc.fit(df_input[[att]])
    #         encnames = [att + "_" + str(i) for i in range(len(enc.categories_[0]))]
    #         print('New columns:', encnames)
    #         df_enc = pd.DataFrame(enc.transform(df_input[[att]]), columns=encnames)
    #         df_input = pd.concat([df_input, df_enc], axis=1)
    #         df_input = df_input.drop([att], axis=1)
    #         inputnames = [i for i in inputnames if i != att] + encnames

    # x_all = df_input[inputnames].values.copy()
    # print("Input shape:", x_all.shape)


    # # for cvind in range(len(train_indices)):
    # for cvind in range(len(train_indices)):
    #     train_index = train_indices[cvind]
    #     test_index = test_indices[cvind]

    #     train_index_allsample = df_basinid['basin_id'].isin(train_index).values
    #     test_index_allsample = df_basinid['basin_id'].isin(test_index).values
        
    #     print('Train/test model')
    #     print('Train index:', train_index)
    #     print('Test index:', test_index)

    #     if objfunc == 'normKGE':
    #         print('Use normalized KGE as output')
    #         df_output = df_metric.copy()
    #         y_all = df_output[["kge"]].values.copy()
    #         y_all = y_all / (2 - y_all)  # Normalize KGE
        
    #         # Train a random forest emulator
    #         outfile = f'{outpath}/RF_emulator_for_iter{iterend}_CVFold{cvind}.pkl'
    #         outfile_eval = f'{outpath}/RF_emulator_for_iter{iterend}_CVFold{cvind}_eval.npz'
    #         if not os.path.isfile(outfile):
    #             with open(outfile, 'rb') as file:
    #                 em_model = pickle.load(file)
    #         else:
    #             modelconfig = {'n_estimators': 100, 'random_state': 42, 'max_depth': 40}
    #             em_model = RandomForestRegressor(**modelconfig, n_jobs=ncpus)
    #             em_model.fit(x_all[train_index_allsample], y_all[train_index_allsample])
    #             with open(outfile, 'wb') as file:
    #                 pickle.dump(em_model, file)

    #             # evaluate em_model on testing samples
    #             y_test_pred = em_model.predict(x_all[test_index_allsample])
    #             np.savez_compressed(outfile_eval, y_test_pred=y_test_pred, y_test=y_all[test_index_allsample], basin_id=df_basinid['basin_id'].values[test_index_allsample]) 
    
    #     elif not objfunc == 'norm2err':
    #         print('Use normalized mae and mmae as output')
    #         # normalization is performed for each basin
    #         df_output = df_metric.copy()
    #         metvalues = df_output[['mae', 'max_mon_abs_err']].values.copy()
    #         y_all = np.nan * metvalues
    #         for i in range(len(df_basin_info)):
    #             indi = df_basinid['basin_id'].values==i
    #             di = metvalues[indi, :]
    #             di = (di - np.nanmin(di,axis=0) ) / (np.nanmax(di,axis=0) - np.nanmin(di,axis=0) )
    #             y_all[indi, :] = di
            
    #         # Train a random forest emulator
    #         outfile = f'{outpath}/RF_emulator_2errOBJfunc_for_iter{iterend}_CVFold{cvind}.pkl'
    #         outfile_eval = f'{outpath}/RF_emulator_2errOBJfunc_for_iter{iterend}_CVFold{cvind}_eval.npz'
    #         if os.path.isfile(outfile):
    #             with open(outfile, 'rb') as file:
    #                 em_model = pickle.load(file)
    #         else:
    #             modelconfig = {'n_estimators': 200, 'random_state': 42, 'max_depth': 40}
    #             em_model = RandomForestRegressor(**modelconfig, n_jobs=ncpus)
    #             em_model.fit(x_all[train_index_allsample], y_all[train_index_allsample])
    #             with open(outfile, 'wb') as file:
    #                 pickle.dump(em_model, file)
                    
    #             # evaluate em_model on testing samples
    #             y_test_pred = em_model.predict(x_all[test_index_allsample])
    #             np.savez_compressed(outfile_eval, y_test_pred=y_test_pred, y_test=y_all[test_index_allsample], basin_id=df_basinid['basin_id'].values[test_index_allsample]) 
                
        
    #     # param_names = df_param_info['Parameter'].values
    #     # parallel_process_basins(df_basinid[test_index_allsample], df_param_lb.iloc[test_index], df_param_ub.iloc[test_index], x_all[test_index_allsample], df_input[test_index_allsample], y_all[test_index_allsample], param_names, inputnames, em_model, inpath_moasmo, ncpus, numruns, iterend, test_index, suffix)

File exists: /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO_bigrange/LSE_spaceCV_PredictParam/camels_627basin_attribute.pkl
The number of attributes used: 27
['mean_elev', 'mean_slope', 'area_gauges2', 'p_mean', 'pet_mean', 'aridity', 'p_seasonality', 'frac_snow', 'high_prec_freq', 'high_prec_dur', 'low_prec_freq', 'low_prec_dur', 'frac_forest', 'lai_max', 'lai_diff', 'dom_land_cover', 'dom_land_cover_frac', 'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity', 'soil_conductivity', 'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac', 'carbonate_rocks_frac', 'geol_permeability']


In [9]:
# path_CTSM_case_all = f'/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_MOASMO_bigrange'
# iter_end =  1
# for tarbasin in target_index:
#     config_file = f'{path_CTSM_case_all}/configuration/_level1-{tarbasin}_config_MOASMO.toml'
#     config = toml.load(config_file)
    
#     # inputs
#     file_parameter_list = config['file_calib_param']
#     path_CTSM_base = config['path_CTSM_case']
#     path_script_MOASMO = config['path_script_MOASMO']
#     path_CTSM_source = config['path_CTSM_source']
#     ref_streamflow = config['file_Qobs']
    
#     if 'add_flow_file' in config:
#         add_flow_file = config['add_flow_file']
#     else:
#         add_flow_file = 'NA'
    
#     script_singlerun = f'{path_script_MOASMO}/run_one_paramset_Derecho.py'
#     script_clone = f'{path_CTSM_source}/cime/scripts/create_clone'

#     if config['path_calib'] == 'NA':
#         path_MOASMOcalib = f'{path_CTSM_base}_MOASMOcalib'
#     else:
#         path_MOASMOcalib = config['path_calib']
        
#     # outputs
#     path_paramset = f'{path_MOASMOcalib}/param_sets_{suffix}'
#     path_submit = f'{path_MOASMOcalib}/run_model_{suffix}'
#     path_archive = f'{path_MOASMOcalib}/ctsm_outputs_{suffix}'
        
#     os.makedirs(path_MOASMOcalib, exist_ok=True) 
    
#     # evaluation period
#     RUN_STARTDATE = config['RUN_STARTDATE']
#     ignore_month = config['ignore_month']
#     STOP_OPTION = config['STOP_OPTION']
#     STOP_N = config['STOP_N']
    
#     if 'nonstandard_evaluation' in config:
#         nonstandard_evaluation = config['nonstandard_evaluation']
#     else:
#         nonstandard_evaluation = 'NA'
    
#     # HPC job settings
#     job_mode = config['job_mode']
#     job_CTSMiteration = config['job_CTSMiteration']
#     # job_controlMOASMO = config['job_controlMOASMO'] # not needed here
    
#     date_start = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(months=ignore_month)).strftime('%Y-%m-%d') # ignor the first year when evaluating model
#     if STOP_OPTION == 'nyears':
#         date_end = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(years=STOP_N)).strftime('%Y-%m-%d')
#     elif STOP_OPTION == 'nmonths':
#         date_end = (pd.Timestamp(RUN_STARTDATE) + pd.offsets.DateOffset(months=STOP_N)).strftime('%Y-%m-%d')
#     else:
#         sys.exit(f'STOP_OPTION must be nyears or nmonths. {STOP_OPTION} is not accepted.')

#     # generate submission commands (note, this won't submit a real job on Derecho)
#     run_multiple_paramsets_Derecho.generate_and_submit_multi_CTSM_runs(iter_end, path_submit, path_paramset, path_CTSM_base, 
#                                                                        path_archive, script_singlerun, script_clone, 
#                                                                        date_start, date_end, ref_streamflow, add_flow_file,
#                                                                        job_CTSMiteration, job_mode)
